In [1]:
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
from scipy.stats import linregress
from influenza_USA.NC_forecasts.utils import get_NC_influenza_data, compute_WIS, simulate_geometric_random_walk, get_historic_drift

In [2]:
# settings
start_baseline_month = 12 # expressed as Hubverse reference date
start_baseline_day = 1
end_baseline_month = 4
end_baseline_day = 7
drift_horizon = 2
seasons = ['2014-2015','2015-2016', '2016-2017', '2017-2018', '2018-2019', '2019-2020', '2023-2024']

In [ ]:
# LOOP seasons
collect_seasons=[]
for focal_season in seasons:
    ## get the current season's data
    data = 7*get_NC_influenza_data(datetime(int(focal_season[0:4]), start_baseline_month, start_baseline_day) - timedelta(weeks=1),
                                    datetime(int(focal_season[0:4])+1, end_baseline_month, end_baseline_day)+timedelta(weeks=4),
                                    focal_season)['H_inc']
    ## LOOP weeks
    collect_weeks=[]
    for date in data.index[:-4]:
        ### GET historical drift 
        mu, sigma = get_historic_drift(focal_season, seasons, date, drift_horizon)
        ### SIMULATE baseline model
        simout = simulate_geometric_random_walk(mu, 0.375, date, data[date], n_sim=1000, n_weeks=4)
        ### COMPUTE WIS score
        collect_weeks.append(compute_WIS(simout, data))
    ## CONCATENATE WEEKS
    collect_weeks = pd.concat(collect_weeks, axis=0)
    collect_weeks = collect_weeks.reset_index()
    collect_weeks['season'] = focal_season
    collect_seasons.append(collect_weeks)
# CONCATENATE SEASONS
collect_seasons = pd.concat(collect_seasons, axis=0)


2014-11-29 00:00:00 0.049895254538298524
2014-12-06 00:00:00 0.020390421900112334
2014-12-13 00:00:00 0.019763273050210122
2014-12-20 00:00:00 0.04179353162858058
2014-12-27 00:00:00 0.08539173616234148
2015-01-03 00:00:00 0.061206025536955914
2015-01-10 00:00:00 -0.023137371241113147
2015-01-17 00:00:00 -0.021551956642760856
2015-01-24 00:00:00 0.013834400101222477
2015-01-31 00:00:00 0.03260362959365247
2015-02-07 00:00:00 0.029432774600471025
2015-02-14 00:00:00 0.020172924004576393
2015-02-21 00:00:00 0.014990209601484073
2015-02-28 00:00:00 0.0026530263976175055
2015-03-07 00:00:00 -0.018658420451540154
2015-03-14 00:00:00 -0.008459334608805304
2015-03-21 00:00:00 -0.03759834707734048
2015-03-28 00:00:00 -0.04951854055855371
2015-04-04 00:00:00 -0.006155260907383672
2015-11-28 00:00:00 0.06615660925345175
2015-12-05 00:00:00 0.043625602488115194
2015-12-12 00:00:00 0.027222551887923902
2015-12-19 00:00:00 0.06512650288804915
2015-12-26 00:00:00 0.08736218089266141
2016-01-02 00:00

In [75]:
np.mean(collect_seasons.groupby(by=['season'])['WIS'].mean())

204.6917323852688